In [26]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [27]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [28]:
#split line into segments of 30 ft
def split_line_at_intervals(line, interval=30):
    distance = 0
    segments = []

    while distance < line.length:
        sub_line = line.interpolate(distance)
        next_distance = distance + interval
        next_sub_line = line.interpolate(next_distance)
        segment = LineString([sub_line, next_sub_line])
        segments.append(segment)
        distance = next_distance

    return segments

def split_geodataframe(geodf, local_crs, interval=30):
    
    geodf = geodf.to_crs(local_crs)
    
    new_rows = []

    for index, row in geodf.iterrows():
        line = row.geometry
        splitted_lines = split_line_at_intervals(line, interval)

        for segment_id, segment in enumerate(splitted_lines):
            new_row = row.copy()
            new_row.geometry = segment
            new_row['segment_id'] = segment_id
            new_rows.append(new_row)

    df = gpd.GeoDataFrame(new_rows, columns=geodf.columns.append(pd.Index(['segment_id'])))
    #add a column that combines street_edge_id and segment_id
    df['street_edge_segment_id'] =  df['street_edge_id'].astype(str) + '_' +  df['segment_id'].astype(str)
    df.crs = local_crs
    return df

In [29]:
#read in seattle validation labels
seattle_labels = gpd.read_file('data/round-2/seattle-validation-labels-round2-230608.geojson')
seattle_labels = seattle_labels.drop(columns=['time_created'])
seattle_labels.head()

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,4292,60,Ballard,NoCurbRamp,47.678276,-122.401039,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40104 47.67828)
1,Devon Snyder,4293,60,Ballard,NoCurbRamp,47.678257,-122.401031,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40103 47.67826)
2,Devon Snyder,4294,56,Ballard,NoCurbRamp,47.678185,-122.401009,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40101 47.67818)
3,Devon Snyder,4295,56,Ballard,NoCurbRamp,47.678165,-122.401031,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40103 47.67817)
4,Devon Snyder,4296,58,Ballard,NoCurbRamp,47.678291,-122.401169,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40117 47.67829)


In [30]:
#count the cluster numbers for each label type
def count_clusters(labels, label_type_id):
    df = cluster_label_type_at_index(labels,label_type_id)[2]
    df["clustered"] = df.duplicated(subset="cluster_id", keep=False)
    df1 = df[df['clustered'] == True]
    return df1.cluster_id.nunique()

In [31]:
cluster_count_seattle = {}
for i in [0,1,2,3,4,5,6,7,8,9]:
    clusters =  count_clusters(seattle_labels, i)
    cluster_count_seattle[label_types[i]] = clusters

In [32]:
cluster_count_seattle 

{'CurbRamp': 330,
 'NoSidewalk': 53,
 'Problem': 408,
 'Occlusion': 14,
 'SurfaceProblem': 288,
 'Obstacle': 85,
 'Other': 0,
 'NoCurbRamp': 114,
 'Crosswalk': 83,
 'Signal': 35}

In [ ]:
#count the numbers for eanch label type
seattle_labels.label_type.value_counts()

In [ ]:
count_clusters(seattle_labels, 0)

In [ ]:
#read in seattle streets
seattle_streets = gpd.read_file("data/round-2/seattle-streets-230608.geojson")

In [ ]:
#obtain cluster to street segment mapping
def cluster_to_street_segment_mapping(df,streets_seg):
     # Group by cluster_id and combine all geometries in each group into a MultiPoint
     df_grouped = df.groupby('cluster_id')['geometry'].apply(lambda x: x.unary_union)
     # Calculate centroid of each group
     centroids = df_grouped.centroid
     # Convert centroids to GeoDataFrame
     cluster_centroids = gpd.GeoDataFrame(geometry=centroids).reset_index()
     cluster_centroids.crs = {'init' :'epsg:4326'}
     cluster_centroids_ft = cluster_centroids.to_crs(epsg=streets_seg.crs.to_epsg())
     #map gdf_centroids to seattle_split, delete index_right column
     cluster_centeroids_street = cluster_centroids_ft.sjoin_nearest(streets_seg, distance_col='distance')
     cluster_centeroids_street = cluster_centeroids_street.drop(columns=['index_right'])
     #select only cluster_id and street_edge_segment_id
     cluster_centeroids_street = cluster_centeroids_street[['cluster_id','street_edge_segment_id']]
     #if there are multiple street_edge_segment_ids for a cluster_id, select the first one
     cluster_centeroids_street = cluster_centeroids_street.groupby('cluster_id').first().reset_index()
     return cluster_centeroids_street

In [ ]:
#construct agreement table
def irr_table (df,street_seg):
    mikey = df[df['username'] == 'mikey']
    devon = df[df['username'] == 'Devon Snyder']
    irr_mikey = mikey.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    irr_devon = devon.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    #replace usernames with 1
    irr_mikey  = irr_mikey .replace('mikey',1)
    irr_devon  = irr_devon .replace('Devon Snyder',1)
    #group by street_edge_segment_id and count the number of label_ids
    irr_mikey = irr_mikey.groupby('street_edge_segment_id').sum()
    irr_devon = irr_devon.groupby('street_edge_segment_id').sum()
    irr_mikey = irr_mikey[['username']].reset_index()
    irr_devon = irr_devon[['username']].reset_index()
    irr_mikey = irr_mikey.rename(columns={'username':'mikey'})
    irr_devon = irr_devon.rename(columns={'username':'devon'})
    irr_mikey_devon = irr_mikey.merge(irr_devon, on='street_edge_segment_id', how='left').fillna(0)
    irr_mikey_devon = irr_mikey_devon.drop(columns=['street_edge_segment_id'])
    
    return irr_mikey_devon

In [ ]:
def krippendorff_alpha(labels,label_type_id, streets, local_crs):
    #clustering by label type
    label_type = cluster_label_type_at_index(labels,label_type_id)[2]
    #split streets into segments
    street_seg = split_geodataframe(streets, local_crs, interval=30)
    #map cluster_id to street_edge_segment_id
    mapping = cluster_to_street_segment_mapping(label_type,street_seg)
    #update the street_edge_segment_id with the one closest to the cluster centroid
    label_type = label_type.merge(mapping, on='cluster_id', how='left')
    #select only the columns we need
    df = label_type[['username','cluster_id','street_edge_segment_id','label_id']]
    #construct agreement table
    irr = irr_table(df,street_seg)
    data_matrix = irr.T.to_numpy()
    alpha = krippendorff.alpha(data_matrix,level_of_measurement='ratio')

    return alpha.round(3)
    

In [ ]:
alpha_seattle = {}
for i in [0,1,2,4,5,7,8,9]:
    alpha = krippendorff_alpha(seattle_labels,i, seattle_streets, 2285)
    alpha_seattle[label_types[i]] = alpha

In [22]:
#30 ft
alpha_seattle

{'CurbRamp': 0.939,
 'NoSidewalk': 0.878,
 'Problem': 0.528,
 'SurfaceProblem': 0.483,
 'Obstacle': 0.527,
 'NoCurbRamp': 0.741,
 'Crosswalk': 0.941,
 'Signal': 0.541}

In [ ]:
for i in [0,1,2,4,5,7,8,9]:
    labels= map_clusters(seattle_labels,i)
    map = seattle_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"], popup=["street_edge_id"])
    map.save("html/seattle_"+label_types[i]+"_230608.html")


In [23]:
#take out label type = surfaceproblem and obstacle
seattle_problems= seattle_labels[(seattle_labels['label_type'] == 'SurfaceProblem') | (seattle_labels['label_type'] == 'Obstacle') ]
#take out severity ==1
seattle_problems = seattle_problems[seattle_problems['severity'] >= 4]
seattle_problems

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
107,Devon Snyder,4401,104,Delridge,SurfaceProblem,47.537506,-122.362396,7rtGfmwUxo5KmMttHSV9Ww,None,4.0,f,"cracks,very broken",None,POINT (-122.36240 47.53751)
188,Devon Snyder,4489,128,Delridge,SurfaceProblem,47.545330,-122.362129,Kc8w6pcDom-DMiFu8X6f1g,None,4.0,f,height difference,None,POINT (-122.36213 47.54533)
215,Devon Snyder,4516,126,Delridge,Obstacle,47.541550,-122.360901,aBEy1ylatfdw-tBVOYQnVA,None,4.0,t,"vegetation,narrow",None,POINT (-122.36090 47.54155)
216,Devon Snyder,4517,126,Delridge,Obstacle,47.541466,-122.360878,s84HXlE9_iPAxg_KeDAbeA,None,4.0,t,"vegetation,narrow",None,POINT (-122.36088 47.54147)
219,Devon Snyder,4520,126,Delridge,Obstacle,47.541306,-122.360840,VThyMOgQ5S--MQ23pkrmgA,None,4.0,t,"vegetation,narrow",None,POINT (-122.36084 47.54131)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2820,mikey,7549,653,Rainier Valley,SurfaceProblem,47.545109,-122.267113,5SzoEruKYsDYRb7iXVQyqQ,None,4.0,f,narrow sidewalk,None,POINT (-122.26711 47.54511)
2850,mikey,7579,678,Rainier Valley,Obstacle,47.544842,-122.264229,pvy64U07YbjfB91SqMugMw,None,4.0,f,trash/recycling can,None,POINT (-122.26423 47.54484)
2966,mikey,7704,754,Windermere,Obstacle,47.661606,-122.274765,Vdkot60btTZ1XjhygbDtZw,None,4.0,f,parked bike,None,POINT (-122.27477 47.66161)
3037,mikey,7776,826,Windermere,SurfaceProblem,47.666527,-122.274193,vKQdmFVRV_A4CUd0PagIJw,None,4.0,f,None,None,POINT (-122.27419 47.66653)


In [24]:
alpha_seattle_problems = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(seattle_problems,i, seattle_streets, 3435)
    alpha_seattle_problems[label_types[i]] = alpha

In [25]:
alpha_seattle_problems

{'Problem': 0.498, 'SurfaceProblem': 0.214, 'Obstacle': 0.64}

In [ ]:
for i in [2,4,5]:
    labels= map_clusters(seattle_problems,i)
    map = seattle_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/seattle_wo_sev1_"+label_types[i]+"_230608.html")